In [3]:
# Force reload of modules to get latest code changes
import sys
import importlib

# Remove cached modules
modules_to_reload = [m for m in sys.modules.keys() if m.startswith('utils.') or m.startswith('resources.')]
for module_name in modules_to_reload:
    del sys.modules[module_name]

print("Modules cleared. Run cells below to reload with latest code.")

Modules cleared. Run cells below to reload with latest code.


In [4]:
# read the dictionary file into a list of all words
from resources.rule_definitions import Rule
rule = Rule('resources/twl06_scrabble_dic_american.txt')
words = rule.scrabble_dictionary

In [5]:
# simple EDA
total_words = len(words)
print('Total words:', total_words)

Total words: 178691


In [6]:
# test simple pattern generator works
from utils.linear.simple_pattern_generator import SimplePatternGenerator

spg = SimplePatternGenerator()
spg.generate(pattern='H_T', deck='HTOIAU-', word_list=words)

['HAT', 'HET', 'HIT', 'HOT', 'HUT']

In [7]:
# test dynamic pattern generator works
from utils.linear.dynamic_pattern_generator import DynamicPatternGenerator

dpg = DynamicPatternGenerator(spg)
dpg.generate(pattern='D(0,2)G', deck='pigsqywozdb-', word_list=words)

['DUG', 'DAG', 'DOG', 'DING', 'DIG', 'DONG']

In [8]:
# test game state can print pretty board 
from utils.matrix.game_state import Game

game = Game(rule)
game.empty_board()
score = game.new_move([('H', [7, 7]), ('O', [7, 8]), ('T', [7, 9])])
print(score)
game.print_board()

12
        0       1       2       3       4       5       6       7       8       9      10      11      12      13      14
  ---------------------------------------------------------------------------------------------------------
 0 |.TW     .       .       .DL     .       .       .       .TW     .       .       .       .DL     .       .       .TW    
 1 |.       .DW     .       .       .       .TL     .       .       .       .TL     .       .       .       .DW     .      
 2 |.       .       .DW     .       .       .       .DL     .       .DL     .       .       .       .DW     .       .      
 3 |.DL     .       .       .DW     .       .       .       .DL     .       .       .       .DW     .       .       .DL    
 4 |.       .       .       .       .DW     .       .       .       .       .       .DW     .       .       .       .      
 5 |.       .TL     .       .       .       .TL     .       .       .       .TL     .       .       .       .TL     .      
 6 |.       .       .DL

In [14]:
from utils.players.aggregate_recommender import AggregateRecommender
from tests.test_aggregate_recommender import _print_moves_dict

ar = AggregateRecommender(rule, game=game)
res = ar.recommend_next_move(deck='HTOIAU-')
_print_moves_dict(game, res, 2)

Recommendations (by optimiser):
- Crossword: 4 move(s)
    1. OHIA (Horizontal) - Score: 27
       Positions: [(6, 7), (6, 8), (6, 9), (6, 10)]
       Additions: [('O', [6, 7]), ('H', [6, 8]), ('I', [6, 9]), ('A', [6, 10])]
    2. OHIAS (Horizontal) - Score: 27
       Positions: [(6, 7), (6, 8), (6, 9), (6, 10), (6, 11)]
       Additions: [('O', [6, 7]), ('H', [6, 8]), ('I', [6, 9]), ('A', [6, 10]), ('s', [6, 11])]
    ... and 2 more moves
- LongestWord: 1 move(s)
    1. OHIAS (Horizontal) - Score: 27
       Positions: [(6, 7), (6, 8), (6, 9), (6, 10), (6, 11)]
       Additions: [('O', [6, 7]), ('H', [6, 8]), ('I', [6, 9]), ('A', [6, 10]), ('s', [6, 11])]
- PrizeCells: 4 move(s)
    1. OHIA (Horizontal) - Score: 27
       Positions: [(6, 7), (6, 8), (6, 9), (6, 10)]
       Additions: [('O', [6, 7]), ('H', [6, 8]), ('I', [6, 9]), ('A', [6, 10])]
    2. OHIAS (Horizontal) - Score: 27
       Positions: [(6, 7), (6, 8), (6, 9), (6, 10), (6, 11)]
       Additions: [('O', [6, 7]), ('H', [6, 